# 0. Import/Classes/Logger

In [1]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (TimeoutException,
                                        ElementNotVisibleException, 
                                        ElementNotSelectableException, 
                                        NoSuchElementException, 
                                        ElementNotInteractableException, 
                                        StaleElementReferenceException, 
                                        ElementClickInterceptedException, 
                                        InvalidSelectorException, 
                                        WebDriverException)

import re
from time import sleep as time_sleep, time
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
import lxml
from itertools import chain
from typing import NamedTuple
import pickle
from collections import deque
from tqdm import tqdm
import pandas as pd
import numpy as np
import logging
from pathlib import Path
import psutil

In [2]:
# получение пользовательского логгера и установка уровня логирования
short_logger = logging.getLogger('short_logger')
short_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
short_handler = logging.FileHandler("data/short_logger.log", encoding='utf-8', mode='a')
short_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
short_handler.setFormatter(short_formatter)
# добавление обработчика к логгеру
short_logger.addHandler(short_handler)

# получение пользовательского логгера и установка уровня логирования
long_logger = logging.getLogger('long_logger')
long_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
long_handler = logging.FileHandler("data/long_logger.log", encoding='utf-8', mode='w')
long_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
long_handler.setFormatter(long_formatter)
# добавление обработчика к логгеру
long_logger.addHandler(long_handler)

In [3]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H:%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + "_" + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')
    

def occupied_memory() -> float:
    # Получить список всех процессов
    all_processes = psutil.process_iter()

    # Пройтись по каждому процессу и найти процессы с именем "chrome.exe"
    chrome_processes = [p for p in all_processes if p.name() == "chrome.exe"]

    # Инициализировать переменную для хранения общего объема памяти
    total_memory = 0

    # Получить информацию о памяти для каждого процесса Chrome

    if chrome_processes:
        for process in chrome_processes:
            try:
                memory_info = process.memory_info()
            except psutil.NoSuchProcess:
                memory_usage = 0
            else:
                memory_usage = memory_info.rss
                total_memory += memory_usage

    # Преобразовать размер в удобочитаемый формат
    # formatted_size = psutil._common.bytes2human(total_memory)
    return (total_memory) / (1024**2)

In [4]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    su1: str
    su2: str
    su3: str
    page_href: str

class MyTimeoutErrore(Exception):
    def __init__(self, text: str='timeout'):
        self.txt = text

class WebDriverMemoryOut(Exception):
    def __init__(self, text: str='Web Driver Memory Out'):
        self.txt = text

In [5]:
driver = webdriver.Chrome()
driver.maximize_window()

# 1. Получение адресов городов

# 2. Получение ссылок на Магазины

## 2.1. Получение ссылки на старицы каталогов с магазинами

### 2.1.1. Функции

In [6]:
def _get_page_count(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### 2.1.2. Выполнение

In [7]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [8]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## 2.2. Получение непосредственно все ссылки на Магазины

### 2.2.1. Функции

In [9]:
def get_retailers_links(driver: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, False, href, None, None, None, None))
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise MyTimeoutErrore('timeout')
    
    return out_list

### 2.2.2. Выполнение

In [10]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
# print(retailer_links)

59


### 2.2.3. Save/Load

In [11]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)

In [12]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

In [13]:
retailer_links[1]

market_link(sity='lipetsk', title='Магнит Косметик', discounts=3528, alcohol=False, href='https://edadeal.ru/lipeck/retailers/magnit-cosmo', su1=None, su2=None, su3=None, page_href=None)

# 3. Получение адресов Товаров

## 3.1. Получение ссылок на каталоги с товарами

### 3.1.1. Функции

In [14]:
def reload_driver(driver: webdriver.Chrome) -> webdriver.Chrome:
    if driver:
        driver.quit()
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver


def _check_load_page(driver: webdriver.Chrome, timeout: int=20, limitMB: int=1400) -> bool:
    """
       Проверка наличия названия продукта в ячейке
    """
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        n = driver.find_elements(By.XPATH, """//div[contains(@class, "p-dsk-srch-retailer__content")]
                                                //div[contains(@class, "b-dsk-grid__container")]
                                                /a[contains(@class, "p-dsk-srch-retailer__card")]
                                                //div[contains(@class, "b-srch-card__price-new")]
                                                //span[@data-test-ref="money-base"]
                                                """)
        
        # Проверка на переполнение памяти:
        if occupied_memory() > limitMB:
            raise WebDriverMemoryOut('Превышение лимита')
            
        if len(n) > 0:
            if int(n[0].text.replace('\xa0', '').replace(' ', '')) > 0:
                return True
        else:
            time_sleep(0.2)
            continue
    return False


                
                
def get_link_to_pages_without_except(driver: webdriver.Chrome, market: market_link, effort: int=10, timeout: int=20, limitMB: int=1400):
    """
         Функция получает ссылки на каталоги товаров в каждом магазине,
         В детализации до SU1/SU2/SU3, если таковые присутствуют.

         Для того что бы обойти ошибки, которые возникают при сборе информации,
         присутствует подфункция, которая перезапускается и начинает сбор данных с последней
         "точки сохранения".
    """
                        
    out_list = list()                # Лист в который собираются данные из подфункции
    HEAD = None                      # Точка сохранения, она опирается на группу SU1
    reload_driver_bool = False       # Флаг обозначающий, что был перезагружен драйвер selenium
    
    def _get_link_to_pages(driver: webdriver.Chrome, market: market_link, timeout: int=20, limitMB: int=1400):
        """
            ПодФункция которая непосредственно исследует страницу с магазином,
            Проходит по категориям SU1, SU2 и достает ссылки на категории SU3, если есть.

            В процессе возникает множество ошибок,
            Которые отлавливаются либо внутри, этой функции, либо во внешней функции.
            
        """

        # Модуль, который позволяет дождаться когда прогрузится необъодимый элемент
        wait = WebDriverWait(
            driver,
            timeout=timeout,
            poll_frequency=1
            )

        alcohol = False       # Рудимент, необъодимо убрать

        # Проброска переменных из внешней функции
        nonlocal out_list
        nonlocal HEAD
        nonlocal reload_driver_bool
        
        # Находим все SU1 категории, исключая первую: "ВСЕ"
        el1 = (driver
              .find_element(By.CSS_SELECTOR, '.p-dsk-srch-retailer__body')
              .find_element(By.CSS_SELECTOR, '.p-dsk-srch-retailer__sidebar-container')
              .find_element(By.CSS_SELECTOR, '.i-block-helper')
              .find_element(By.CSS_SELECTOR, '.b-dsk-srch-cats-tree__node')
              .find_elements(By.XPATH, "following-sibling::*")
              )
        
        # Активируем режим +18
        if not alcohol:
            for _ in range(20):
                try:
                    driver.find_element(By.XPATH, '//div[contains(text(), "Да. Мне есть 18")]/ancestor::button').click()
                except NoSuchElementException:
                    time_sleep(0.2)
                    continue
                except ElementNotInteractableException:
                    time_sleep(0.2)
                    continue
                else:
                    alcohol = True
                    break
        
        # Цикл по каждой SU1 категории отдельно
        for el in el1:
            
            # Временный лист, в который будет собираться информация о SU2 и SU3
            temp_list = list()
            
            # Кликаем на категории, что бы открыть список подкатегорий SU2
            el2 = (el
                    .find_element(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__item-wrapper")]/button')
                    )
            su1 = el2.text
            
            # Проверка перезагрузки драйвера
            # И продолжение с места остановки (SU1)
            if not HEAD:
                reload_driver_bool = False
            
            if reload_driver_bool:
                if not (HEAD == su1):
                    continue
                else:
                    reload_driver_bool = False

            # Продолжение с места остановки (SU1)
            el2.click()
            time_sleep(0.2)

            # Проверка, что загрузились товары, а так же проверка на превышения лимита объема оперативной памяти.
            check_page = _check_load_page(driver, timeout, limitMB)
            
            if check_page:
                
                # Фиксация "точки сохранения", возможно стоило бы перенести точку сохранения выше, это нужно проверить.
                HEAD = su1

                # Находим ссылки на SU2, (в будующем необходимо переписать название переменных)
                try:
                    el3 = (el
                        .find_element(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__children")]/div[contains(@class, "i-block-helper")]')
                        .find_elements(By.XPATH, './/div[contains(@class, "b-dsk-srch-cats-tree__node")]//button')
                        )
                except NoSuchElementException:
                    # Редко, но бывает, что подкатегорий нет.
                    short_logger.info(f'only su1: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                    href = driver.current_url
                    su2 = np.nan
                    su3 = np.nan
                    temp_list.append(market_link(
                                                    market.sity,
                                                    market.title,
                                                    market.discounts,
                                                    market.alcohol,
                                                    market.href,
                                                    su1,
                                                    su2,
                                                    su3,
                                                    href
                                                ))
                    continue
                    
            else:
                raise MyTimeoutErrore()         # Информация на страницу так и не загрузиласть

            # SU2
            for item in el3:
                # Проходим по каждой  SU2
                for _ in range(10):
                    try:
                        item.click()
                    except ElementClickInterceptedException:
                        # Иногда категория не открывается, потому что она не видна на экране
                        body = driver.find_element(By.XPATH, '//body')
                        body.send_keys(Keys.ARROW_DOWN)
                        time_sleep(0.2)
                        continue
                    else:
                        break
                else:
                    # Иногда, что-то еще.
                    raise ElementClickInterceptedException()

                # С после нажатия на категорию, мы прогручиваем минисписок вниз на один элемент, что бы показалась следующая SU2
                time_sleep(0.2)
                item.send_keys(Keys.ARROW_DOWN)

                # Проверяем загрузку товаров при выборе категории SU2
                check_page = _check_load_page(driver, timeout, limitMB)
                                
                if check_page:
                    # Достаем SU2
                    su2 = item.text

                    # Достаем список с категориями SU3,
                    su3_list = wait.until(lambda d: d.find_elements(By.XPATH, '//li[contains(@class, "b-dsk-srch-cats-list__item")]/button'))
    
                    # Если 3 и более елементов в списке, это означает, что у категории SU2 есть подкатегории SU3.
                    if len(su3_list) > 2:
                        for s_item in su3_list[1:]:
                            su3 = s_item.text
                            # Таким способом мы добывает ссылки на каталог с детализацией до SU3
                            # Потом еще предстоит пройтись по каждой странице каталога, но если добавить эту операцию сюда,
                            # это сильно уложнит выполнения кода, и точки сохранения, так как малое количество памяти накладывает свои ограничения.
                            href = driver.current_url + '&segmentUuid2='+ s_item.get_attribute('value')
                            temp_list.append(market_link(
                                                            market.sity,
                                                            market.title,
                                                            market.discounts,
                                                            market.alcohol,
                                                            market.href,
                                                            su1,
                                                            su2,
                                                            su3,
                                                            href
                                                        ))
                    else:
                        su3 = np.nan
                        href = driver.current_url
                        temp_list.append(market_link(
                                                        market.sity,
                                                        market.title,
                                                        market.discounts,
                                                        market.alcohol,
                                                        market.href,
                                                        su1,
                                                        su2,
                                                        su3,
                                                        href
                                                    ))
            
            # Добавляем информацию во внешний лист
            out_list.append(temp_list)

        # Оповещаем внешнюю функцию, что в этом магазине закончились категории SU1
        HEAD = 'finally'
        # ====================================================================== Конец

    # Цикл который будет пытаться занова начать собирать информацию, пока не истечет количество попыток
    # Что делать когда исзасходуется effort???????
    for _ in range(effort):
        # Закончились ли категории SU1
        if not (HEAD == 'finally'):
            # Перезагрузка драйвера
            driver = reload_driver(driver)
            try:
                # Открытие ссылки с магазином
                driver.get(market.href)
            except WebDriverException as ex:
                short_logger.error(f'WebDriverException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.error(f'WebDriverException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue

            # Первая загрузка магазина особенно долгая, по этому закладываем дополнительное время на проверку.
            if not _check_load_page(driver, timeout=120):
                short_logger.error(f'WebDriver TimeOut: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.error(f'WebDriver TimeOut: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue
            
            try:
                driver.find_element(By.XPATH, '//p[contains(text(), "В этом магазине сейчас нет акций")]')
            except NoSuchElementException:
                pass
            else:
                short_logger.warning(f'В этом магазине сейчас нет акций: ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                continue
            
            reload_driver_bool = True
            try:
                _get_link_to_pages(driver, market, timeout, limitMB)
            # except WebDriverException as ex:
            #     print('_get_link_to_pages WebDriverException', ex)
            #     continue
            except WebDriverMemoryOut as ex:
                short_logger.info(f'WebDriverMemoryOut: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                continue
            except StaleElementReferenceException as ex:
                short_logger.warning(f'StaleElementReferenceException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.warning(f'StaleElementReferenceException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue
            except ElementClickInterceptedException as ex:
                short_logger.error(f'ElementClickInterceptedException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}')
                long_logger.error(f'ElementClickInterceptedException: {ex} ::  market: {market.title} ::  HEAD: {HEAD} ::  href: {market.href}', exc_info=True)
                continue
            else:
                driver.quit()
        else:
            return out_list
                     

### 3.1.2. Выполнение

In [15]:
dead_line = 20
out_list = list()

market_deque = deque(retailer_links)

with tqdm(total=len(market_deque)) as pbar:
    while len(market_deque) > 0:
        if not dead_line:
            break
        
        market = market_deque.popleft()
        
        if market.discounts > 0:
            # print("Маркет: ", market.title)
            out = get_link_to_pages_without_except(driver, market)
            
            if out is None:
                market_deque.append(market)
                dead_line -= 1
                continue
            
            out_list.extend(out)
        pbar.update(1)

100%|██████████| 59/59 [52:37<00:00, 53.52s/it]  


In [16]:
market_catalog_links = list(chain.from_iterable(out_list.copy()))
print(f'Всего ссылок: {len(market_catalog_links)}')
for item in market_catalog_links:
    if not isinstance(item, market_link):
        print('несовпало')


Всего ссылок: 3592


In [77]:
market_catalog_links[-1000]

market_link(sity='lipetsk', title='Mi-shop.com', discounts=200, alcohol=False, href='https://edadeal.ru/lipeck/retailers/mi-shopcom', su1='Бытовая техника', su2='Мелкая техника для кухни', su3='Прочая техника', page_href='https://edadeal.ru/lipeck/retailers/mi-shopcom?segmentUuid=8903c83b-8001-5157-bb31-2e7e7e56d64e&segmentUuid2=9e9f8c55-f5e4-590a-ad33-35a01e94c964')

### 3.1.3. Save/Load

Ссылки на страницы с информацией

In [17]:
with open('data/market_catalog_links.pickle', 'wb') as file:
    pickle.dump(market_catalog_links, file)

In [73]:
with open('data/market_catalog_links.pickle', 'rb') as file:
    market_catalog_links = pickle.load(file)

## 3.2. Получение информации о товарах и ссылки на них

### 3.2.1 Функции

#### 3.2.1.1 Парсинг информации

In [124]:
def parser_sku_01(page_source: str, market: market_link, date: datetime) -> pd.DataFrame:
    """
        Извлечение данных из страницы,
        Данные возвращаются в виде pd.DataFrame
    """
    
    # Получаем хост из ссылки
    host = '/'.join(market.page_href.split('/', maxsplit=3)[:3])
    # 
    page = BS(page_source, 'lxml')
    
    # Находим карточки с продуктами
    f1 = page.find('div', {'data-test-ref':'grid'})
    f2 = f1.find('div').find_all('a', {'data-test-ref':'b-srch-card'})
    
    out_list = list()
    
    for f3 in f2:
        # название
        title = f3.find('div', class_='b-srch-card__title').get_text(strip=True)
        
        # дата до истечения
        try:
            date_to = datetime.strptime(f3.find('time').get('datetime'), '%Y-%m-%d')
        except AttributeError:
            date_to = np.nan
        
        # Цена (разбита на части)
        try:
            b = (f3
                .find('div', class_="b-srch-card__price")
                .find('div', class_="b-srch-card__price-new")
                .find('div', class_='b-money__root-wrapper')
                .find('span', {'data-test-ref':"money-base"})
                .get_text(strip=True)
                .replace('\xa0', '')
                )
        except AttributeError as ex:
            price = np.nan
            short_logger.critical(f"no price, {ex}: {market.title} :: {market.page_href}")
            long_logger.critical(f"no price, {ex}: {market.title} :: {market.page_href}", exc_info=True)
        
        else:
            try:
                s = (f3
                    .find('div', class_="b-srch-card__price")
                    .find('div', class_="b-srch-card__price-new")
                    .find('div', class_='b-money__root-wrapper')
                    .find('span', {'data-test-ref':"money-subunit"})
                    .get_text(strip=True)
                    .replace('\xa0', ''))
            except AttributeError:
                s = '0'
                
            price = float(f'{b}.{s}')
        
        # -------------------------------------------------------------------------------------------------
        # Предыдущая цена (разбита на части)
        try:
            b_last = (f3
                      .find('div', class_="b-srch-card__price")
                      .find('div', class_="b-srch-card__price-old")
                      .find('div', class_='b-money__root-wrapper')
                      .find('span', {'data-test-ref':"money-base"})
                      .get_text(strip=True)
                      .replace('\xa0', '')
                      )
        except AttributeError as ex:
            price_last = np.nan
        
        else:
            try:
                s_last = (f3
                          .find('div', class_="b-srch-card__price")
                          .find('div', class_="b-srch-card__price-old")
                          .find('div', class_='b-money__root-wrapper')
                          .find('span', {'data-test-ref':"money-subunit"})
                          .get_text(strip=True)
                          .replace('\xa0', '')
                          )
            except AttributeError:
                s_last = '0'
                
            price_last = float(f'{b_last}.{s_last}')

        # -------------------------------------------------------------------------------------------------
        # тип валюты
        try:
            unit = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"currency"}).get_text(strip=True)
        except AttributeError:
            unit = np.nan

        # -------------------------------------------------------------------------------------------------
        # скидка
        try:
            sale = f3.find('div', class_='b-srch-card__discount-label').get_text(strip=True)
        except AttributeError:
            sale = np.nan
        
        # -------------------------------------------------------------------------------------------------
        # Условие
        try:
            condition = (f3
                         .find('div', class_="b-srch-card__content-container")
                         .find('div', class_="b-tpl-offer-label__text")
                         .get_text(strip=True)
                         .replace('\xa0', '')
                         )
        except AttributeError as ex:
                condition = np.nan

        # -------------------------------------------------------------------------------------------------
        # вес
        try:
            f4 = (f3
                .find('div', class_='b-srch-card__info')
                .find('span')
                .get_text(strip=True)
                .replace('\xa0', ' ')
                .split('•')
                )
            vol = f4[0].strip()
        
        # руб/кг. руб/шт.
            vol_vs = f4[1].strip()
        except AttributeError:
            vol = np.nan
            vol_vs = np.nan
                
        # -------------------------------------------------------------------------------------------------
        # ссылка
        try:
            href = host + f3.get('href')
        except AttributeError as ex:
            href = np.nan
        
                
        out_list.append((date, market.sity, market.title, market.su1, market.su2, market.su3, title, date_to, condition ,price, price_last, unit, sale, vol, vol_vs, href))


    return pd.DataFrame(out_list, columns=['date', 'sity', 'market', 'SU1', 'SU2', 'SU3', 'title', 'date_to', 'condition', 'price', 'price_last', 'unit', 'sale', 'vol', 'vol_vs', 'href'])

#### 3.2.1.2 Получение страницы

In [27]:
def _get_page(driver: webdriver.Chrome, item: market_link, timeout: int=30) -> webdriver.Chrome.page_source:
    """
        сохраняем страницы
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    
    except TimeoutException as ex:
        short_logger.warning(f"don't get response: {item.title} :: {item.page_href}")
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    except Exception as ex:
        short_logger.critical(f"other exception, {ex}: {item.title} :: {item.page_href}")
        long_logger.critical(f"other exception, {ex}: {item.title} :: {item.page_href}", exc_info=True)
    
    # Проверка сайта на 18 лет
    # Мы ранее собирали информацию, и приблизительно занем есть ли в данном магазине такая проверка или нет.
    if item.alcohol:
        try:
            # Ожидаем появляение кнопки, если она не повится, то ничего не делаем.
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        
        except TimeoutException:
            short_logger.info(f'alcochol timeout: {item.title} :: {item.page_href}')
        except ElementNotInteractableException:
            short_logger.warning(f'alcochol not button: {item.title} :: {item.page_href}')
        except Exception as ex:
            short_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}')
            long_logger.error(f'alcochol other: {ex} :: {item.title} :: {item.page_href}', exc_info=True)
    
    # Ручной тайм аут на выполнение всего процесса
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout*2:
        
        # Задержка действий до загрузки отслеживаемых елементов.
        # Варант провекрки загрузки страницы, проверка что текст не пустой
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@data-test-ref='grid']"))
        except TimeoutException:
            try:
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                short_logger.warning(f"not table, timeout: {item.title} :: {item.page_href}")
                raise MyTimeoutErrore()
            else:
                short_logger.info(f"not page, timeout: {item.title} :: {item.page_href}")
                page = None
                break
            
        # Проверка наличия названия продукта в ячейке
        page = BS(driver.page_source, 'lxml')
        
        try:
            # Дожидаемся когда загрузится информация о первом товаре на странице
            f1 = page.find('div', {'data-test-ref':'grid'})
            f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
            f2[0].find('div', class_='b-srch-card__title').get_text(strip=True)
        except IndexError as ex:
            continue
        except Exception as ex:
            short_logger.critical(f"checking the product name, other exception, {ex}: {item.title} :: {item.page_href}")
            long_logger.critical(f"checking the product name, other exception, {ex}: {item.title} :: {item.page_href}", exc_info=True)

        break
    
    else:
        short_logger.warning(f"not table, timeout ALL: {item.title} :: {item.page_href}")
        raise MyTimeoutErrore()
    
    return driver.page_source

#### 3.2.1.3. Получение информации с сайта.

In [28]:
def get_pages(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=20, dead_line: int=20) -> tuple[pd.DataFrame, list[market_link]]:
    """
        Функция получает страницы с информацией об товарах,
        извлекает информацию,
        отдает датафрейм с информацией и необработанными ссылками.
    """
    table = list()
    
    with tqdm(total=len(list_url)) as pbar:     # Прогресс бар
        
        q = deque(list_url)                     # Очередь
        out_list = list()
        
        while len(q) > 0:
            if dead_line < 0:                   # Ограничение количества попыток получить данные.
                return (out_list, q)
            
            item = q.popleft()
            if item.discounts:
                try:
                    page = _get_page(driver, item, timeout)
                except MyTimeoutErrore:
                    q.append(item)
                    dead_line -= 1
                    continue
                else:
                    table.append(parser_sku_01(page, item, datetime.now()))
                
            pbar.update(1)
        df = pd.concat(table, ignore_index=True)
    return (df, list(q))                        # Возвращаем отработаный лист и очередь(пустую или нет)

#### 3.2.1.4 Новый вариант

In [78]:
driver = webdriver.Chrome()
driver.maximize_window()

In [127]:
df_list = list()
for market in market_catalog_links[1800:1820]:
    driver.get(market.page_href)
    _check_load_page(driver)
    find = driver.find_elements(By.XPATH, """//nav[contains(@class, "b-dsk-paginator")]
                                             /a[contains(@class, "b-dsk-paginator__item")]
                                                    """)
    df_list.append(parser_sku_01(driver.page_source, market, datetime.now()))
    if len(find):
        for n in range(2, len(find)):
            link = f'?page={n}&'.join(market.page_href.split('?'))
            driver.get(link)
            _check_load_page(driver)
            df_list.append(parser_sku_01(driver.page_source, market, datetime.now()))

In [128]:
pd.concat(df_list, ignore_index=True).to_clipboard(decimal=',', index=False)

### 3.2.2. Выполнение

#### 3.2.2.1. Сохранение всех страниц, для того что бы отточить работу парсера.

In [19]:
# pages, other = _get_pages(driver, retailer_links_pages)

100%|██████████| 914/914 [35:32<00:00,  2.33s/it]  


In [20]:
# Сохранение страниц  в Zip файл.
with ZipFile('data/pages.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
    for index, page in enumerate(pages):
        if page:
            zip_file.writestr(f'{index:05d}.html', page.encode())

In [6]:
# Поллучение страниц из Zip файла.
pages = list()
with ZipFile('data/pages.zip', 'r') as zf:
    for item in zf.filelist:
        with zf.open(item.filename) as file:
            pages.append(file.read())

In [7]:
url = retailer_links_pages[0].page_href


In [23]:


dataframe_list = list()

for page in pages[:200]:
    dataframe_list.append(parser_sku_01(page, retailer_links_pages[0], datetime.strptime("2023-08-28", '%Y-%m-%d')))
out_df = pd.concat(dataframe_list, ignore_index=True)

(24, 13)